# Evaluación de nuestro RAG

In [9]:
from langchain_mistralai import ChatMistralAI
from langchain_core.output_parsers import StrOutputParser

from classes.chatbot import Chatbot
from classes.LLM import LLM, EMBEDDING

import pandas as pd
from tqdm import tqdm
import time
import warnings
from dotenv import load_dotenv
import os
import re

load_dotenv()
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")
warnings.filterwarnings("ignore", category=FutureWarning)

El modelo que vamos a usar para las evaluaciones y nuestro RAG

In [10]:
# modelo de lenguaje que usaremos para evaluar
llm = ChatMistralAI(
    model="mistral-small-latest", 
    mistral_api_key=MISTRAL_API_KEY,
    temperature=0, 
    random_seed=12345
)

search_type = "similarity" # ["similarity", "mmr", "tfidf", "bm25", "grafo"]
chroma_directory = "chroma/nomic_512"  

chatbot = Chatbot(
    # language_model=LLM.llama_3_2_3B.value,   # modelo de lenguaje a testear
    chroma_directory=chroma_directory,      # directorio de la base de datos
    search_type=search_type,                # tipo de búsqueda en la base de datos
    embedding_model=EMBEDDING.NOMIC,         # misma funcion de embedding que use la base de datos
    k=10,                                   # documentos recuperados de la base de datos
    top_n=5                                 # documentos recuperados por el reranker
)

Cargamos una muestra de nuestro dataset

In [ ]:
df = pd.read_csv("eval/test - preguntas_mistral_plus.csv", on_bad_lines="skip")
df_shuffle = df.sample(frac=1, random_state=12345)
# test_sample = df_shuffle.head(1)
test_sample = df
# df_shuffle.head(10)

## Configuración de los agentes críticos

- **Accuracy**: Grado de precisión de la respuesta generada
- **Faithfulness**: Si la respuesta se basa exclusivamente en los documentos recuperados
- **Groundedness**: Si la pregunta puede ser respondida con el contexto aportado
- **Relevance**: Si los docs recuperados tienen que ver con la pregunta

In [12]:
accuracy_prompt = """
Eres un agente crítico experto en evaluación de precisión de respuestas. 
Evalúa la precisión de la respuesta generada en comparación con la respuesta esperada. 

Criterios de evaluación:
- 5: La respuesta generada incluye toda la información esencial de la respuesta esperada, incluso si es más extensa o utiliza otras palabras.
- 4: La respuesta generada incluye casi toda la información importante, con leves omisiones o diferencias menores que no afectan el significado general.
- 3: La respuesta generada cubre parte significativa del contenido, pero falta información importante o hay errores leves.
- 2: La respuesta generada solo refleja una parte limitada del contenido de la respuesta esperada y tiene omisiones o distorsiones relevantes.
- 1: La respuesta generada tiene muy poca relación con la respuesta esperada, presenta errores graves u omisiones importantes.
- 0: La respuesta generada no contiene información relevante de la respuesta esperada o está completamente incorrecta.

Recuerda: una redacción diferente o más extensa no debe penalizar si toda la información está presente.

respuesta esperada: {respuesta_esperada}

respuesta generada: {respuesta_generada}

Responde solo con tu evaluación en el siguiente formato:

Respuesta:::
Rating: (tu evaluación del 0 al 5)
Respuesta:::
"""

faithfulness_prompt = """
Tu tarea es evaluar si la respuesta generada usa EXCLUSIVAMENTE información contenida en los documentos proporcionados, sin añadir datos externos, suposiciones infundadas ni invenciones.

Criterios de evaluación:
5 - Totalmente basada en los documentos, sin añadir nada externo.
4 - Principalmente basada, con pequeñas generalizaciones o frases neutras.
3 - Incluye inferencias plausibles pero no explícitas en los documentos.
2 - Mezcla partes basadas en los documentos con invenciones o suposiciones.
1 - Mayormente inventada, aunque con algún detalle correcto.
0 - Completamente inventada, contradictoria o irrelevante.

Aquí tienes los documentos recuperados y la respuesta generada por el sistema RAG.

Documentos recuperados: {documentos}

Respuesta generada: {respuesta}

Responde solo con tu evaluación en el siguiente formato:

Respuesta:::
Rating: (tu evaluación del 0 al 5)
Respuesta:::
"""

groundedness_prompt = """
Tu tarea es evaluar en qué medida la respuesta está respaldada por evidencia específica de los documentos proporcionados.

Criterios de evaluación:
5 - Usa varias evidencias clave con precisión y contexto adecuado.
4 - Usa al menos una evidencia clave de forma clara y correcta.
3 - Se refiere a los documentos de manera vaga o genérica.
2 - Uso superficial, indirecto o débilmente relevante de los documentos.
1 - Menciona información incorrecta o mal interpretada del documento.
0 - No hace uso de los documentos; ignora la evidencia disponible.

Aquí tienes los documentos recuperados y la respuesta generada por el sistema RAG.

Documentos recuperados: {documentos}

Respuesta generada: {respuesta}

Responde solo con tu evaluación en el siguiente formato:

Respuesta:::
Rating: (tu evaluación del 0 al 5)
Respuesta:::
"""


relevance_prompt = """
Tu tarea es evaluar qué tan relevantes son los documentos para responder la pregunta.

Criterios de evaluación:
5 - Abordan directamente el núcleo de la pregunta
4 - Cubren aspectos importantes pero no todos
3 - Proporcionan contexto general útil
2 - Tienen conexión tangencial con el tema
1 - Mínima relación con la pregunta
0 - Completamente irrelevantes

Aquí tienes la pregunta y los documentos recuperados.

Pregunta: {pregunta}

Respuesta esperada: {respuesta}

Documentos recuperados: {documentos}

Responde solo con tu evaluación en el siguiente formato:

Respuesta:::
Rating: (tu evaluación del 0 al 5)
Respuesta:::
"""

In [13]:
def get_response(question: str, chatbot: Chatbot):
    results = chatbot.get_retriever().invoke(question)
    context = "\n\n---\n\n".join([doc.page_content for doc in results]) 
    
    response = chatbot.answer_query2(question, context)

    return response, context

def get_response_reranker(question: str, chatbot: Chatbot):
    results = chatbot.get_compression_retriever().invoke(question)
    context = "\n\n---\n\n".join([doc.page_content for doc in results]) 
    
    response = chatbot.answer_query2(question, context)

    return response, context

Evaluación del RAG + Reranking

In [ ]:
columns = [
    "pregunta", "respuesta esperada", "respuesta generada", 
    "accuracy", "faithfulness", 
    "groundedness", "relevance"
]

eval_rag = []
eval_rag_rr = []

sleep = 2

for index, row in tqdm(test_sample.iterrows(), desc="Evaluando el RAG", total=len(test_sample), unit="pregunta"):
    pregunta = row["Pregunta"]
    respuesta = row["Respuesta"]
    contexto = row["Contexto"]       

    ### NORMAL ###

    time.sleep(sleep) # Espera para evitar sobrecargar el servidor
    respuesta_generada, docs_recuperados = get_response(pregunta, chatbot)  
    # print(docs_recuperados)

    evaluations = {
        "accuracy": accuracy_prompt.format(respuesta_esperada=respuesta, respuesta_generada=respuesta_generada),
        "faithfulness": faithfulness_prompt.format(documentos=docs_recuperados, respuesta=respuesta_generada),
        "groundedness": groundedness_prompt.format(documentos=docs_recuperados, respuesta=respuesta_generada),
        "relevance": relevance_prompt.format(pregunta=pregunta, respuesta=respuesta, documentos=docs_recuperados)
    }        


    ### RERANKING ###
    time.sleep(sleep)
    respuesta_generada_rr, docs_recuperados_rr = get_response_reranker(pregunta, chatbot)   
    # print(docs_recuperados)

    evaluations_rr = {
        "accuracy": accuracy_prompt.format(respuesta_esperada=respuesta, respuesta_generada=respuesta_generada_rr),
        "faithfulness": faithfulness_prompt.format(documentos=docs_recuperados_rr, respuesta=respuesta_generada_rr),
        "groundedness": groundedness_prompt.format(documentos=docs_recuperados_rr, respuesta=respuesta_generada_rr),
        "relevance": relevance_prompt.format(pregunta=pregunta, respuesta=respuesta, documentos=docs_recuperados_rr)
    } 

    try:

        eval_row = {
            "pregunta": pregunta,
            "respuesta esperada": respuesta,
            "respuesta generada": respuesta_generada            
        }
        for criterion, evaluation in evaluations.items():
            time.sleep(sleep) 
            chain = llm | StrOutputParser()
            response = chain.invoke(evaluation)
            match = re.search(r'Rating:\s*(\d+)', response)
            score = int(match.group(1))
            eval_row[criterion] = score
        eval_rag.append(eval_row)

        #####################

        eval_row_rr = {
            "pregunta": pregunta,
            "respuesta esperada": respuesta,
            "respuesta generada": respuesta_generada_rr            
        }
        for criterion, evaluation in evaluations_rr.items():
            time.sleep(sleep) 
            chain = llm | StrOutputParser()
            response = chain.invoke(evaluation)
            match = re.search(r'Rating:\s*(\d+)', response)
            score = int(match.group(1))
            eval_row_rr[criterion] = score
        eval_rag_rr.append(eval_row_rr)
            
    except Exception as e:
        continue

###############################
eval_rag_df = pd.DataFrame(eval_rag, columns=columns)
eval_rag_df.to_csv(f"eval/eval_{search_type}.csv", index=False)
eval_rag_df_rr = pd.DataFrame(eval_rag_rr, columns=columns)
eval_rag_df_rr.to_csv(f"eval/eval_{search_type}_rr.csv", index=False)

Evaluando el RAG: 100%|██████████| 1/1 [00:41<00:00, 41.77s/pregunta]
